# Rozwiązanie: Implementacja prostej ontologii uniwersytetu w Pythonie

Poniżej przedstawiono implementację ontologii uniwersytetu zgodnie z wymaganiami zadania, wykorzystując bibliotekę `owlready2`.

In [2]:
from owlready2 import *

# Tworzenie ontologii
onto = get_ontology("http://example.org/university.owl")

with onto:
    class Person(Thing):
        pass

    class Student(Person):
        pass

    class Lecturer(Person):
        pass

    class Course(Thing):
        pass

    class Department(Thing):
        pass

    # Właściwości obiektowe
    class teaches(ObjectProperty):
        domain = [Lecturer]
        range = [Course]

    class attends(ObjectProperty):
        domain = [Student]
        range = [Course]

    class belongs_to(ObjectProperty):
        domain = [Person, Course]
        range = [Department]

    # Właściwości danych
    class has_name(DataProperty):
        domain = [Person]
        range = [str]

    class has_id(DataProperty):
        domain = [Person, Course]
        range = [str]

    # Reguła: Jeśli student uczęszcza na kurs, a wykładowca uczy tego kursu,
    # to student jest uczony przez tego wykładowcę
    class is_taught_by(ObjectProperty):
        domain = [Student]
        range = [Lecturer]

    AllDisjoint([Student, Lecturer, Course, Department])

    # Implementacja reguły
    rule = Imp()
    rule.set_as_rule("""
        Student(?s), Course(?c), Lecturer(?l),
        attends(?s, ?c), teaches(?l, ?c)
        -> is_taught_by(?s, ?l)
    """)

# Tworzenie instancji
with onto:
    student1 = Student("student1")
    student1.has_name = "Alice"
    student1.has_id = "S001"

    student2 = Student("student2")
    student2.has_name = "Bob"
    student2.has_id = "S002"

    lecturer1 = Lecturer("lecturer1")
    lecturer1.has_name = "Dr. Smith"
    lecturer1.has_id = "L001"

    course1 = Course("course1")
    course1.has_id = "C001"

    dept1 = Department("dept1")

    student1.attends.append(course1)
    student2.attends.append(course1)
    lecturer1.teaches.append(course1)
    student1.belongs_to.append(dept1)
    student2.belongs_to.append(dept1)
    lecturer1.belongs_to.append(dept1)
    course1.belongs_to.append(dept1)

# Synchronizacja reasonera
sync_reasoner()

# Funkcja do przeszukiwania ontologii
def search_ontology(onto):
    print("Studenci:")
    for student in onto.Student.instances():
        print(f"  - {student.has_name} (ID: {student.has_id})")
        print(f"    Kursy: {[course.has_id for course in student.attends]}")
        print(f"    Nauczany przez: {[lecturer.has_name for lecturer in student.is_taught_by]}")

    print("\nWykładowcy:")
    for lecturer in onto.Lecturer.instances():
        print(f"  - {lecturer.has_name} (ID: {lecturer.has_id})")
        print(f"    Uczy kursów: {[course.has_id for course in lecturer.teaches]}")

    print("\nKursy:")
    for course in onto.Course.instances():
        print(f"  - Kurs {course.has_id}")
        print(f"    Studenci: {[student.has_name for student in course.attends.indirect()]}")
        print(f"    Wykładowca: {[lecturer.has_name for lecturer in course.teaches.indirect()]}")

# Wywołanie funkcji przeszukującej
search_ontology(onto)


ModuleNotFoundError: No module named 'owlready2'

## Objaśnienie rozwiązania

1. Zdefiniowano klasy `Person`, `Student`, `Lecturer`, `Course` i `Department`.
2. Utworzono właściwości obiektowe `teaches`, `attends` i `belongs_to`.
3. Dodano właściwości danych `has_name` i `has_id`.
4. Zaimplementowano regułę wnioskowania dla relacji `is_taught_by`.
5. Utworzono instancje klas i ustanowiono relacje między nimi.
6. Zaimplementowano funkcję `search_ontology` do przeszukiwania i wyświetlania informacji z ontologii.

To rozwiązanie demonstruje podstawowe koncepcje tworzenia ontologii, w tym definiowanie klas, właściwości, relacji i reguł wnioskowania. Funkcja `search_ontology` pokazuje, jak można przeszukiwać i analizować utworzoną ontologię.